In [2]:
"""
Data Preprocessing
"""

import os
import re
import pandas as pd

# read train data
file_dir = '/Users/dnhb/PycharmProjects/SML_Ass1'
file_name = "train.txt"

def parse(data:list)->list: 
    # Python Notation: https://www.python.org/dev/peps/pep-3107/
    """
    Prase each line of the data
    Return a nested list, such as [[id1, id2, id2,...],...]
    """
    parsed_data = []
    for line in data:
        # remove \n at the end of each line
        line = re.sub(r"(?<=\d)\n", "", line)
        # split IDs by \t
        pattern = re.compile("(?<=\d)\t(?=\d)")
        line = re.split(pattern, line)
        parsed_data.append(line)
    return parsed_data

with open(os.path.join(file_dir, file_name)) as f:
    train_set = f.readlines()
    train_set = parse(train_set)
    
# read test data
test_name = "test-public.txt"
with open(os.path.join(file_dir, test_name)) as f:
    test_set = f.readlines()
    test_set = parse(test_set)
    test_set.pop(0)

# turn ID into int
for i in range(0, len(test_set)):
    test_set[i][0] = int(test_set[i][0])
    
print("Data has been read.")

Data has been read.


In [3]:
"""
Sigma Data For Similarity
"""

from collections import defaultdict

train_dic = {}
for item in train_set:
    train_dic[item[0]] = item[1:]

# construct SigmaOut dic for each node
SigmaOut = defaultdict(set)

def getSigmaOut(node:str, graph:dict) -> set:
    SigmaOut = set()
    try:
        for item in graph[node]:
            SigmaOut.add(item)
        return SigmaOut
    except:
        return SigmaOut

for kw in train_dic:
    SigmaOut[kw] = getSigmaOut(kw, train_dic)
    
print("SigmaOut done.\n")

# construct SigmaIn dic for each node
SigmaIn = defaultdict(set)

for kw in train_dic:
    for followee in train_dic[kw]:
        if SigmaIn[followee]:
            SigmaIn[followee].add(kw)
        else:
            SigmaIn[followee] = {kw}

def getSigma(node:str) -> set:
    return SigmaIn[node].union(SigmaOut[node])

print("SigmaIn done.")

SigmaOut done.

SigmaIn done.


In [5]:
"""
Similarity Functions
"""

import math

    
def adar1(a:str, b:str) -> float:
    score = 0
    SigmaA = getSigma(a)
    SigmaB = getSigma(b) 
    SigmaV = SigmaA.intersection(SigmaB)
    if SigmaV:
        for v in SigmaV:
            if getSigma(v) and len(getSigma(v)) != 1:
                   score += (1/math.log(len(getSigma(v))))
            else:
                   pass
        return score
    else:
        return 0

def adar2(a:str, b:str) -> float:
    score = 0
    SigmaOutA = SigmaOut[a]
    if SigmaOutA:
        for node in SigmaOutA:
            score += adar1(b, node)
        return float(score/len(SigmaOutA))
    else:
        return 0
    
def adar3(a:str, b:str) -> float:
    score = 0
    SigmaInB = SigmaIn[b]
    if SigmaInB:
        for node in SigmaInB:
            score += adar1(a, node)
        return float(score/len(SigmaInB))
    else:
        return 0
  
def jaccard1(a:str, b:str) -> float:
    SigmaA = getSigma(a)
    SigmaB = getSigma(b)
    return float(len(SigmaA.intersection(SigmaB)) / len(SigmaA.union(SigmaB)))

def jaccard2(a:str, b:str) -> float:
    score = 0
    SigmaOutA = SigmaOut[a]
    if SigmaOutA:
        for node in SigmaOutA:
            score += jaccard1(b, node)
        return float(score/len(SigmaOutA))
    else:
        return 0

def jaccard3(a:str, b:str) -> float:
    score = 0
    SigmaInB = SigmaIn[b]
    if SigmaInB:
        for node in SigmaInB:
            score += jaccard1(a, node)
        return float(score/len(SigmaInB))
    else:
        return 0
    
def cosine1(a:str, b:str) -> float:
    SigmaA = getSigma(a)
    SigmaB = getSigma(b)
    return float(len(SigmaA.intersection(SigmaB)) / (len(SigmaA)*len(SigmaB)))


def cosine2(a:str, b:str) -> float:
    score = 0
    SigmaOutA = SigmaOut[a]
    if SigmaOutA:
        for node in SigmaOutA:
            score += cosine1(b, node)
        return float(score/len(SigmaOutA))
    else:
        return 0

def cosine3(a:str, b:str) -> float:
    score = 0
    SigmaInB = SigmaIn[b]
    if SigmaInB:
        for node in SigmaInB:
            score += cosine1(a, node)
        return float(score/len(SigmaInB))
    else:
        return 0

def common_neighbors1(a: str, b: str) -> float:
    SigmaA = getSigma(a)
    SigmaB = getSigma(b)
    return len(SigmaA.intersection(SigmaB))

def common_neighbors2(a: str, b: str) -> float:
     score = 0
     SigmaOutA = SigmaOut[a]
     if SigmaOutA:
         for node in SigmaOutA:
             score += common_neighbors1(b, node)
         return float(score / len(SigmaOutA))
     else:
         return 0

def common_neighbors3(a: str, b: str) -> float:
     score = 0
     SigmaInB = SigmaIn[b]
     if SigmaInB:
         for node in SigmaInB:
             score += common_neighbors1(a, node)
         return float(score / len(SigmaInB))
     else:
         return 0

def knn1(a:str, b:str) -> float:
    """
    knn1 = $w_{ab} = w_a^out$
    """
    return 1/math.sqrt(1+len(SigmaOut[a]))

def knn2(a:str, b:str) -> float:
    """
    knn2 = $w_{ab} = w_b^in$
    """
    return 1/math.sqrt(1+len(SigmaIn[b]))

def knn3(a:str, b:str) -> float:
    """
    knn3 = $w_{ab} = w_a^out  w_b^in$
    """
    return 1/math.sqrt(1+len(SigmaOut[a])) + 1/math.sqrt(1+len(SigmaIn[b]))

def knn4(a:str, b:str) -> float:
    """
    knn4 = $w_{ab} = w_a^out * w_b^in$
    """
    wAOut = 1/math.sqrt(1+len(SigmaOut[a]))
    wBIn = 1/math.sqrt(1+len(SigmaIn[b]))
    return  wAOut* wBIn

In [6]:
"""
Construct Train Data
Output:
1. train data: positive, negative
2. label of train data: train_label
"""

import random

train_dic = {}
for item in train_set:
    train_dic[item[0]] = item[1:]

# collect user list in the original train set
user_list = list(train_dic.keys())
random.seed(0)
# select 200 user randomly
user_train = random.sample(user_list, 200)

# construct positive dataset
    # the element of positive_list = the link that comes from traversing the adjacent list of one user
positive_list = []
sink_set = set()
for user in user_train:
    for sink in train_dic[user]:
        if (user, sink) not in positive_list:
            positive_list.append((user, sink))
        sink_set.add(sink)
print("The length of positive list is", len(positive_list), ".\n")
print("The length of sink set is", len(sink_set), ".\n")

# construct negative dataset
    # the element (user1 -> user2) of negative_list = 
    # user1 = an element in the user list
    # user2 = any element except user1's adjacency
negative_list = []

count=0
for user in user_train:
    for sink in sink_set:
        if (user, sink) not in positive_list:
            if (user, sink) not in negative_list:
                negative_list.append((user, sink))
    count=count+1
    if count%20==0:
        print(count)
print("The length of negative list is", len(negative_list), ".")

# construct train dataset
random.seed(0)
positive_train = random.sample(list(positive_list), 10000)
random.seed(0)
negative_train = random.sample(list(negative_list), 3000)
# create label for positive and negative train data
positive_label = [1] * len(positive_train)
negative_label = [0] * len(negative_train)
# combine positive and negative train data & label
#train_link = []
#train_link.extend(positive_train)
#train_link.extend(negative_train)
Y_train = []
Y_train.extend(positive_label)
Y_train.extend(negative_label)

del user_list, user_train
del positive_list, sink_set, negative_list, count
#del positive_train, negative_train, positive_label, negative_label

The length of positive list is 171315 .

The length of sink set is 144184 .

20
40
60
80
100
120
140
160
180
200
The length of negative list is 28665485 .


In [20]:
"""
Get Feature
"""

def getFeature(pair:tuple) -> float:
    node1 = pair[0]
    node2 = pair[1]
    result = []
    result.append(adar1(node1,node2))
    #result.append(salton(node1,node2))
    #result.append(allocation(node1,node2))
    result.append(jaccard1(node1,node2))
    result.append(jaccard2(node1,node2))
    result.append(jaccard3(node1,node2))
    result.append(cosine1(node1,node2))
    result.append(cosine2(node1,node2))
    result.append(cosine3(node1,node2))
    result.append(knn1(node1,node2))
    result.append(knn2(node1,node2))
    result.append(knn3(node1,node2))
    result.append(knn4(node1,node2))
    result.append(common_neighbors1(node1,node2))
    result.append(common_neighbors2(node1,node2))
    result.append(common_neighbors3(node1,node2))
    result.append(AA(node1,node2))
    #result.append(CNPA(node1,node2))
    print(result)

    return result

In [8]:
"""

"""

import multiprocessing as mp
import time
    
if __name__ == '__main__':
         
    # construct the # of pools corresponding to the cpu_count in ur PC
    pool = mp.Pool(mp.cpu_count())
    
    startTime = time.time()
    
    positive = pool.map(getFeature, positive_train)
    pool.close()
    pool.join()
    
    endTime = time.time()
    print("Total time:" + (endTime - startTime).__str__())
    
    pool = mp.Pool(mp.cpu_count())
    
    startTime = time.time()
    
    negative = pool.map(getFeature, negative_train)
    pool.close()
    pool.join()
    
    endTime = time.time()
    print("Total time:" + (endTime - startTime).__str__())
    
    X_train = []
    X_train.extend(positive)
    X_train.extend(negative)

[0.0, 0.008972309321891575, 0.5, 0.5089723093218915, 0.004486154660945788, 0, 1.3227598422027211, 5712.666666666667]
[0.00031213421771361687, 0.008972309321891575, 0.24253562503633297, 0.25150793435822455, 0.00217610464940429, 4, 1.9975042267128251, 1634.4375]
[0.0, 0.016850210249810097, 0.7071067811865475, 0.7239569914363576, 0.011914897932059788, 0, 1.0, 3552.0]
[0.0040297582145071295, 0.008972309321891575, 0.08058229640253803, 0.08955460572442961, 0.0007230092891919219, 52, 19.997665244344255, 1703.078431372549]
[0.0002243549794341269, 0.004999000299900035, 0.24253562503633297, 0.247534625336233, 0.001212435662293071, 9, 2.7350243658628015, 6861.5625]
[0.0014903129657228018, 0.02774568328089315, 0.1796053020267749, 0.20735098530766805, 0.004983271825604053, 2, 2.5554699537750385, 114.1]
[4.985541928407618e-05, 0.004999000299900035, 0.30151134457776363, 0.30651034487766365, 0.0015072553019675032, 2, 1.8630513557415969, 7480.2]
[0.0, 0.015992325525180033, 0.5773502691896258, 0.5933425

[0.00022433261048381066, 0.004999000299900035, 0.2182178902359924, 0.22321689053589241, 0.001090871298733279, 9, 2.0732475321754342, 4160.85]
[0.0, 0.004999000299900035, 0.5, 0.5049990002999001, 0.0024995001499500177, 0, 1.528901661876796, 20452.666666666668]
[0.0011697070754834375, 0.004999000299900035, 0.09090909090909091, 0.09590809120899095, 0.000454454572718185, 47, 8.611820567287268, 2881.175]
[0.0, 0.004999000299900035, 0.5773502691896258, 0.5823492694895259, 0.002886174168826306, 0, 1.3400974634512057, 26887.5]
[0.0007473469184395397, 0.004999000299900035, 0.12403473458920847, 0.1290337348891085, 0.0006200496754094745, 30, 5.738048231913033, 3599.1875]
[0.0, 0.07537783614444091, 0.35355339059327373, 0.42893122673771467, 0.026650089544451302, 0, 1.7142857142857142, 43.285714285714285]
[0.026569118213323067, 0.022355091700494795, 0.03949762527666821, 0.061852716977163005, 0.000882973035011699, 69, 58.656, 184.3203125]
[0.0361794500723589, 0.04170288281141495, 0.10050378152592121,

[0.0, 0.018405254345807757, 0.7071067811865475, 0.7255120355323552, 0.013014480157383837, 0, 1.0, 2960.0]
[0.0009960407380661868, 0.004999000299900035, 0.10425720702853739, 0.10925620732843742, 0.0005211818092023985, 40, 7.095414219667624, 3132.6153846153848]
[0.015834348355663823, 0.08873565094161139, 0.08304547985373997, 0.17178113079535134, 0.007369094712580091, 13, 58.32539682539682, 27.67361111111111]
[0.0, 0.004999000299900035, 0.5773502691896258, 0.5823492694895259, 0.002886174168826306, 0, 1.0224915656628764, 20506.5]
[0.0025430586059415096, 0.010785785934387398, 0.1125087900926024, 0.12329457602698979, 0.0012134957256757352, 22, 4.504363001745201, 496.34615384615387]
[0.0009049773755656109, 0.01774992567234187, 0.08703882797784893, 0.1047887536501908, 0.0015449327272145685, 3, 5.707532303813426, 139.59541984732823]
[0.0005977732944780692, 0.004999000299900035, 0.12309149097933272, 0.12809049127923275, 0.0006153344003208268, 24, 4.970486067724603, 3071.9846153846156]
[0.0, 0.01

[0.061224489795918366, 0.14907119849998599, 0.3779644730092272, 0.5270356715092132, 0.0563436169819011, 3, 3.409090909090909, 26.166666666666668]
[0.0073085478233238, 0.010380684981717497, 0.06984302957695782, 0.08022371455867532, 0.000725018488207177, 69, 33.33301002263175, 1520.8333333333333]
[0.0007660878447395301, 0.015992325525180033, 0.30151134457776363, 0.31750367010294367, 0.0048218675720223215, 3, 2.497058071117933, 976.1]
[0.0012160518848804217, 0.007621605866236445, 0.13483997249264842, 0.14246157835888487, 0.0010276971253531302, 21, 5.0477518299058906, 1612.3333333333333]
[0.01999540335555045, 0.015992325525180033, 0.04343722427630694, 0.05942954980148697, 0.0006946622305369532, 87, 51.94295216167818, 383.82797731568996]
[2.4929573953581132e-05, 0.004999000299900035, 0.3779644730092272, 0.3829634733091272, 0.0018894445139246855, 1, 1.349468949144071, 9025.333333333334]
[0.005564472980203317, 0.010380684981717497, 0.10425720702853739, 0.11463789201025489, 0.00108226122323695

[0.0006226184842975618, 0.004999000299900035, 0.11867816581938533, 0.12367716611928536, 0.0005932721865226934, 25, 4.587229788829189, 2631.4285714285716]
[0.11396011396011396, 0.057259833431386825, 0.14744195615489714, 0.20470178958628396, 0.00844250185022725, 40, 13.361842105263158, 78.24444444444444]
[0.002406832298136646, 0.008972309321891575, 0.09578262852211514, 0.10475493784400672, 0.0008593913707642516, 31, 16.499557201513564, 1963.351851851852]
[0.008623548922056384, 0.01855993633210361, 0.1091089451179962, 0.1276688814500998, 0.0020250550746529965, 26, 3.626119917298415, 132.42168674698794]
[0.00019942664838589058, 0.004999000299900035, 0.25, 0.2549990002999, 0.0012497500749750088, 8, 1.5789578907909534, 4226.133333333333]
[7.478499314470896e-05, 0.004999000299900035, 0.30151134457776363, 0.30651034487766365, 0.0015072553019675032, 3, 1.383081344495814, 5550.7]
[0.00019939185484272969, 0.004999000299900035, 0.20851441405707477, 0.2135134143569748, 0.001042363618404797, 8, 1.32

...

[0.0, 0.017122183231250473, 0.7071067811865475, 0.7242289644177979, 0.0121072118715358, 0, 0.004105571847507331, 14.0]
[0.0, 0.15249857033260467, 0.35355339059327373, 0.5060519609258785, 0.053916386601719206, 0, 0.7380952380952381, 4.571428571428571]
[0.0, 0.08111071056538127, 0.079555728417573, 0.16066643898295427, 0.006452821661495841, 0, 2.543046357615894, 2.8471337579617835]
[0.004201680672268907, 0.13736056394868904, 0.07352146220938077, 0.2108820261580698, 0.010098949511412773, 1, 9.865384615384615, 3.641304347826087]
[0.03140096618357488, 0.06119900613621046, 0.08164965809277261, 0.14284866422898307, 0.0049968779266390765, 13, 25.210526315789473, 49.57046979865772]
[0.008016032064128256, 0.11867816581938533, 0.05773502691896257, 0.17641319273834788, 0.006851887098275315, 4, 12.314285714285715, 2.6454849498327757]
[0.0, 0.05415303610738823, 0.1889822365046136, 0.24313527261200182, 0.010233961877089323, 0, 1.411764705882353, 19.37037037037037]
[0.0, 0.05415303610738823, 0.35355339

[0.0, 0.12216944435630522, 0.04867923835112355, 0.17084868270742876, 0.005947115501044907, 0, 5.257575757575758, 2.2399049881235156]
[0.0, 0.12909944487358055, 0.7071067811865475, 0.836206226060128, 0.09128709291752767, 0, 0.0, 0.0]
[0.0, 0.1643989873053573, 0.2773500981126146, 0.44174908541797187, 0.04559607525875532, 0, 0.19444444444444445, 0.75]
[0.008733624454148471, 0.022321651006080835, 0.12216944435630522, 0.14449109536238605, 0.0027270237005282572, 18, 13.323030907278165, 405.8787878787879]
[0.0, 0.20851441405707477, 0.20851441405707477, 0.41702882811414954, 0.04347826086956522, 0, 0.5454545454545454, 0.5454545454545454]
[0.0, 0.0716114874039433, 0.10314212462587934, 0.17475361202982265, 0.0073861609584621075, 0, 1.9793814432989691, 4.172043010752688]
[0.0, 0.09325048082403138, 0.4472135954999579, 0.5404640763239893, 0.04170288281141495, 0, 0.32456140350877194, 9.75]
[0.0, 0.11952286093343936, 0.5773502691896258, 0.6968731301230652, 0.06900655593423544, 0, 0.10144927536231885, 

[0.0, 0.08192319205190406, 0.08944271909999159, 0.17136591115189564, 0.007327433054473118, 0, 2.2972972972972974, 5.080645161290323]
[0.0, 0.19611613513818404, 0.7071067811865475, 0.9032229163247315, 0.1386750490563073, 0, 0.0, 0.0]
[0.0, 0.0265092566603795, 0.5, 0.5265092566603795, 0.01325462833018975, 0, 0.12728551336146274, 63.333333333333336]
[0.0, 0.04789131426105757, 0.7071067811865475, 0.754998095447605, 0.033864273073929814, 0, 0.0367816091954023, 18.0]
[0.010809312638580931, 0.016835875742536848, 0.09622504486493763, 0.11306092060747448, 0.0016200328986661233, 39, 20.50808052168982, 678.0841121495328]
[0.0, 0.16666666666666666, 0.7071067811865475, 0.8737734478532141, 0.1178511301977579, 0, 0.0, 0.0]
[0.0, 0.0716114874039433, 0.5, 0.5716114874039433, 0.03580574370197165, 0, 0.3865979381443299, 25.666666666666668]
[0.0, 0.5, 0.7071067811865475, 1.2071067811865475, 0.35355339059327373, 0, 0.0, 0.0]
[0.0, 0.12909944487358055, 0.0854357657716761, 0.21453521064525666, 0.011029709933

[0.0, 0.12909944487358055, 0.09853292781642932, 0.22763237269000985, 0.012720546282869609, 0, 1.152542372881356, 1.3431372549019607]
[0.00425531914893617, 0.1889822365046136, 0.07035975447302918, 0.25934199097764277, 0.013296743760228545, 1, 9.37037037037037, 2.3930348258706466]
[0.0, 0.4472135954999579, 0.7071067811865475, 1.1543203766865053, 0.3162277660168379, 0, 0.0, 0.0]
[0.0, 0.1889822365046136, 0.5, 0.6889822365046137, 0.0944911182523068, 0, 0.0, 0.3333333333333333]
[0.0, 0.03167031776097682, 0.21320071635561041, 0.24487103411658723, 0.0067521344338500695, 0, 1.03714859437751, 56.285714285714285]
[0.0, 1.0, 0.7071067811865475, 1.7071067811865475, 0.7071067811865475, 0, 0, 2.0]
[0.0, 0.7071067811865475, 0.10721125348377948, 0.8143180346703269, 0.07580980435789034, 0, 3.0, 0.03488372093023256]
[0.0, 0.07088812050083358, 0.22941573387056174, 0.30030385437139534, 0.01626285018740355, 0, 3.3333333333333335, 36.833333333333336]
[0.0, 0.1543033499620919, 0.4472135954999579, 0.601516945

[0.0, 0.08111071056538127, 0.21320071635561041, 0.29431142692099166, 0.017292861596651866, 0, 1.0529801324503312, 8.857142857142858]
[0.0, 0.10101525445522107, 0.5, 0.6010152544552211, 0.050507627227610534, 0, 0.0, 1.3333333333333333]
[0.0, 0.07254762501100116, 0.08451542547285165, 0.15706305048385283, 0.006131393394849657, 0, 5.682539682539683, 9.136690647482014]
[0.001579778830963665, 0.04163054471218133, 0.14002800840280097, 0.1816585531149823, 0.005829442264770509, 1, 3.3246527777777777, 39.54]
[0.004830917874396135, 0.07432941462471664, 0.2886751345948129, 0.36300454921952957, 0.021457053771143732, 1, 0.8333333333333334, 16.727272727272727]
[0.0, 0.1643989873053573, 0.2773500981126146, 0.44174908541797187, 0.04559607525875532, 0, 0.4722222222222222, 2.0833333333333335]
[0.0, 0.09090909090909091, 0.2672612419124244, 0.35817033282151534, 0.024296476537493127, 0, 0.15833333333333333, 2.076923076923077]
[0.0005544774050457444, 0.016708490237236185, 0.4082482904638631, 0.42495678070109

In [9]:
title = ["jaccard1", "knn1", "knn2", "knn3", "knn4", 
        "common_neighbors1", "common_neighbors2", "common_neighbors3"]
test_pd = pd.DataFrame(columns = title, data = X_train)
test_pd.to_csv('/Users/dnhb/PycharmProjects/SML_Ass1/X_train2Score.csv',encoding='utf-8')

In [19]:
assert(len(X_train) == len(Y_train))

In [18]:
"""
This cell only works when reading data from local 
"""

# read X_train data
score_name = "X_trainScore.csv"
with open(os.path.join(file_dir, score_name)) as f:
    X_train = f.readlines()
    X_train = parse(X_train)
    X_train.pop(0)
    
for i in range(0, len(X_train)):
    X_train[i] = X_train[i][0].split(",")
    for j in range(0, 7):
        X_train[i][j] = float(X_train[i][j])
        
positive = X_train[:10000]
negative = X_train[10000:]
positive = random.sample(positive, 3000)

X_train = []
X_train.extend(positive)
X_train.extend(negative)

# construct Y
positive_label = [1] * 3000
negative_label = [0] * 3000
Y_train = []
Y_train.extend(positive_label)
Y_train.extend(negative_label)

"""
Step:
1. run this cell
2. run the *assert block*

if need to select feature, then
1. run the X_trian1 cell, select the features
2. edit and run *getFeature* cell
3. run the paparmeters of RF
4. gridsearch and so on 
"""

'\nStep:\n1. run this cell\n2. run the *assert block*\n\nif need to select feature, then\n1. run the X_trian1 cell, select the features\n2. edit and run *getFeature* cell\n3. run the paparmeters of RF\n4. gridsearch and so on \n'

In [6]:
# "adar1",  "jaccard2", "jaccard3", "cosine2", "cosine3"
X_train1 = []
for item in X_train:
    new = []
    new.append(item[0])
    new.extend(item[2:4])
    new.extend(item[5:])
    X_train1.append(new)

In [21]:
from sklearn.ensemble import RandomForestRegressor
#from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import numpy as np

# Number of trees in random forest
n_estimators = [50, 100]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [50,70]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 3, 5, 8, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 6, 8]
# Method of selecting samples for training each tree
#bootstrap = [True]
# Create the random grid
para_grid = {'n_estimators': n_estimators,
             #'max_features': max_features,
             'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [22]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np


rf = RandomForestRegressor(bootstrap=True, criterion='mse',
           max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_weight_fraction_leaf=0.0, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False )
grid_search = GridSearchCV(estimator = rf, param_grid = para_grid, cv = 3, verbose=2, n_jobs = -1)
# Fit the random search model
grid_search.fit(X_train, Y_train)
best_grid = grid_search.best_estimator_
print (best_grid)
print ("train success!")

Fitting 3 folds for each of 150 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   21.3s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   51.3s
[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:  1.1min finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=2,
                      min_samples_split=3, min_weight_fraction_leaf=0.0,
                      n_estimators=50, n_jobs=1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)
train success!


In [23]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

rf = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=2,
                      min_samples_split=3, min_weight_fraction_leaf=0.0,
                      n_estimators=50, n_jobs=1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)
rf.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=2,
                      min_samples_split=3, min_weight_fraction_leaf=0.0,
                      n_estimators=50, n_jobs=1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [24]:
import multiprocessing as mp
import time

if __name__ == '__main__':
    
    def getPro(record:list) -> list:
        feature = getFeature((record[1], record[-1]))
        prediction = rf.predict([feature])
        pro = [record[0], prediction[0]]
        print(pro)
        return pro
    
    # construct the # of pools corresponding to the cpu_count in ur PC
    pool = mp.Pool(mp.cpu_count())
    
    startTime = time.time()
    
    result = pool.map(getPro, test_set)
    pool.close()
    pool.join()
    
    endTime = time.time()
    print("Total time:" + (endTime - startTime).__str__())
    
    title = ["Id", "Predicted"]
    test_pd = pd.DataFrame(columns = title, data = result)
    test_pd.to_csv('/Users/dnhb/PycharmProjects/SML_Ass1/RF4Score.csv',encoding='utf-8')

[0, 0.0, 0.000668210329735728, 1.8060941139843826e-05, 0.0, 0.0003432280747153645, 7.228122423328354e-08]
[442, 0.0]
[0.21933342447606208, 0.005154639175257732, 0.0010560980236248867, 0.00576653084902314, 0.001038961038961039, 0.00021586492795896465, 1.5493731237062647e-05]
[316, 0.0]
[0, 0.0, 0.004756335876775658, 0.0005961701829318616, 0.0, 0.0011086098198699584, 3.067381859917711e-05]
[1, 0.0]
[0, 0.0, 0.0004547391940738874, 0.007885478256716273, 0.0, 5.109882367717751e-05, 0.0002286423081153753]
[2, 0.03]
[0.4627293341056383, 0.011152416356877323, 0.00691070154584895, 0.024294678294895098, 0.0006920415224913495, 0.0005541850751437441, 0.000186636503156485]
[3, 0.3269047619047619]
[0.822336943717954, 0.02147239263803681, 0.012449053139136003, 0.006981832564143511, 0.0004018369690011481, 0.0005297209380207122, 3.751707002281268e-05]
[127, 0.07333333333333333]
[0, 0.0, 0.0, 5.02373715807189e-05, 0.0, 0.0, 5.58585883976126e-07]
[128, 0.0]
[1.0980735409705342, 0.0136986301369863, 0.0100

[257, 0.0]
[9.553196002316588, 0.08213820078226858, 0.03708215296325779, 0.015602962057144473, 0.00040492595639654463, 0.0005804369562858414, 7.834012390034121e-05]
[384, 0.946]
[7.2420165395727185, 0.1040339702760085, 0.03509928038377913, 0.03126384551199466, 0.0007985788555876074, 0.0008058146526775432, 0.00016642458555895216]
[134, 0.9833333333333333]
[29.897949237351067, 0.18254764292878636, 0.04679525197987241, 0.06124789248925156, 0.000530893943725242, 0.000823041354766312, 0.0002354258927378306]
[258, 1.0]
[0.9557282025623033, 0.01951219512195122, 0.056342771882562884, 0.04554460018142172, 0.00392156862745098, 0.020121664006326683, 0.0010484155040296956]
[135, 1.0]
[0, 0.0, 0.0007490310165941993, 0.0006286491974390266, 0.0, 9.222095759689693e-05, 5.8356978613035805e-06]
[259, 0.0]
[0.3951550344920891, 0.005319148936170213, 0.009364623952932537, 0.004344694779634571, 8.66494811862314e-05, 0.00027968364738458916, 7.855870264151892e-06]
[327, 0.008]
[0, 0.0, 6.905600441958428e-05, 

[388, 0.99]
[0.8594809787712238, 0.009732360097323601, 0.02493463074241243, 0.016673153206640486, 0.0002264685067232838, 0.0016092657007510261, 2.9935610328027735e-05]
[269, 0.91]
[0, 0.0, 0.005026846856494025, 9.48488334700701e-05, 0.0, 0.00032461637968785273, 5.566130204513274e-07]
[389, 0.0]
[1.0151450146183802, 0.047337278106508875, 0.012987012987012988, 0.0009977427977643364, 0.0010422094841063053, 0.006578947368421052, 1.118810733142825e-05]
[390, 0.06533333333333333]
[0.4222714979851464, 0.008064516129032258, 0.00909297397551057, 0.0004756284174228895, 0.0006491398896462187, 0.00012690003797965002, 3.8384606256878464e-05]
[391, 0.0]
[0.08641664237211644, 0.0005530973451327434, 0.012613703121305359, 0.005033892459930729, 1.2597631645250693e-05, 0.0005804499889526888, 4.828358995037932e-06]
[462, 0.03]
[5.89840889263453, 0.2765957446808511, 0.22433041898233752, 0.23798123932020654, 0.007703703703703704, 0.0054158102023765965, 0.00857107705529758]
[463, 1.0]
[0.09086603972658462, 0

[341, 0.0]
[0.6308496722741597, 0.011326860841423949, 0.029993583843214648, 0.001429203624047571, 0.0002349308632031145, 0.0010066535526959094, 2.514915796436938e-06]
[83, 0.0]
[0.07796806287802142, 0.008620689655172414, 0.002833135675752537, 0.0009194535730035492, 0.0007936507936507937, 0.0003407906870974851, 9.129539416302582e-06]
[342, 0.0]
[0, 0.0, 0.0004662482087038752, 0.011395674325381222, 0.0, 0.00010153730817576423, 2.109094351112229e-05]
[30, 0.0]
[0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[31, 0.0]
[0.0897643439842049, 0.001644736842105263, 0.010781835144750259, 0.003661147501991882, 0.00010539629005059021, 0.0007971638935756517, 6.427931436284521e-06]
[197, 0.0]
[0, 0.0, 9.572530311883386e-05, 0.00045641259698767686, 0.0, 4.8008331060126055e-05, 4.8702879963635184e-06]
[198, 0.0]
[0.9009275289488635, 0.008396305625524769, 0.027762626946808224, 0.005228121079645545, 0.0002384358607534573, 0.0009916955281161007, 4.7446541724281886e-06]
[273, 0.0]
[4.012838591117671, 0.0042897211681240

[404, 0.0]
[0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[405, 0.0]
[0.1348914676310476, 0.034482758620689655, 0.01982248429891781, 0.0005840204864718558, 0.006944444444444444, 0.00477098675850637, 2.4576243681681973e-05]
[406, 0.015]
[0.161171841453854, 0.006622516556291391, 0.008701827784632894, 0.013934745386808011, 0.0008680555555555555, 0.0017439113907804081, 0.00012759601934455027]
[407, 0.11]
[2.900907837240828, 0.028268551236749116, 0.036434856756650945, 0.027347390379633334, 0.0017667844522968198, 0.002796821440145552, 0.0001448851050436918]
[408, 0.9866666666666666]
[1.3336874622678931, 0.008111239860950173, 0.013647456696596243, 0.00418490803129282, 3.570663429265157e-05, 0.00029194023628196554, 3.9288644993157475e-06]
[86, 0.006666666666666666]
[0.25787794265940944, 0.017341040462427744, 0.012018969176871848, 0.0010703735961127346, 0.00038819875776397513, 0.0008250837484596522, 1.2451851372614818e-05]
[279, 0.0]
[0.5475104108019864, 0.008156606851549755, 0.008789623211455435, 0.0177714

...

[1575, 0.95]
[0.14404745418365533, 0.006172839506172839, 0.022798757742688226, 0.005764517618948065, 0.00015220700152207003, 0.0011903124495085727, 7.469637454768008e-05]
[1960, 0.15]
[0.7417729746315271, 0.0215633423180593, 0.03332165247169544, 0.002141590700711793, 0.0003407445267910384, 0.0012797867985949918, 7.901902498809246e-06]
[1799, 0.07033333333333333]
[0.5697119841867475, 0.008032128514056224, 0.040078292155778815, 0.0037914906249670753, 9.775171065493646e-05, 0.001328609845450864, 8.687295225304388e-06]
[1961, 0.0]
[4.356711691040949, 0.07887323943661972, 0.0489928801006575, 0.033868361964878684, 0.0007715198941915574, 0.0012164416647306717, 0.0002915663359211713]
[1962, 1.0]
[0.17180629722095309, 0.00014087483271113616, 0.01503191620966585, 0.014905598687026003, 4.6961585423123884e-05, 0.0063346833679797385, 1.4129988024174388e-06]
[1741, 0.3833333333333333]
[0.26609575091198334, 0.017699115044247787, 0.025054792105840478, 0.012102171724912627, 0.0006747638326585695, 0.001

[1871, 0.8843333333333334]
[0.14817537844738762, 0.027777777777777776, 0.022981830862013494, 0.029130793273379224, 0.014285714285714285, 0.013683945164812566, 0.0008491639174807708]
[1872, 0.9466666666666665]
[0, 0.0, 0.00643854335222437, 0.0013788290304027314, 0.0, 0.000822894318771467, 1.538217983407396e-05]
[1873, 0.0]
[0, 0.0, 0.006097560975609756, 4.710719893574191e-05, 0.0, 3.301298510747561e-05, 1.2824861306519383e-06]
[1812, 0.0]
[0.07734508450738445, 0.0038461538461538464, 0.0052498336175748565, 4.744171225630645e-05, 0.0019305019305019305, 0.0032145087465140026, 1.8329621465277442e-07]
[1813, 0.0]
[0.07383035914027619, 0.008547008547008548, 0.02234642098467787, 6.672345448752019e-05, 0.002898550724637681, 0.009615465646814434, 5.827244817831952e-07]
[1814, 0.0]
[1.6140962003581474, 0.013024602026049204, 0.03177993806610567, 0.0053806520347001625, 0.0006215684243240443, 0.0019911286168679855, 3.9831277232276985e-06]
[1874, 0.10872222222222222]
[58.61633497030306, 0.10083186286

[1880, 0.4786666666666666]
[4.41296319718825, 0.23076923076923078, 0.1434067284395369, 0.12391245320514893, 0.009453781512605041, 0.005505455074952521, 0.0062727177897853904]
[1881, 1.0]
[0.2057705553540774, 0.004415011037527594, 0.005759773288763993, 0.0037137634137854123, 4.8976393378391615e-05, 0.00045464687030952086, 3.438895992094616e-05]
[1825, 0.0]
[0, 0.0, 0.039367150550683704, 0.0003236540939577, 0.0, 0.0012160101616000526, 4.997895828287238e-06]
[1826, 0.0]
[0, 0.0, 0.0029153457339098007, 0.0031830079790381425, 0.0, 0.00022623123372948002, 2.957629434518387e-05]
[1827, 0.0]
[0.07383035914027619, 0.003134796238244514, 0.017479801177151973, 0.0021372603261149608, 0.00010283833813245579, 0.001172309624906079, 1.3903655162717756e-05]
[1882, 0.0]
[0.7672173249735703, 0.004158964879852125, 0.02209482195849002, 0.007563803663559101, 0.00017449976733364356, 0.0016718033146028066, 4.800768747628718e-06]
[1755, 0.04666666666666667]
[0, 0.0, 0.0013750858265679977, 0.007228915662650603, 

[1492, 0.0]
[0.34798632450535943, 0.005208333333333333, 0.045245383025311905, 0.027955690872359523, 0.0006567425569176883, 0.009110714827192267, 6.937141092112767e-05]
[1986, 0.9549999999999998]
[7.571493257276355, 0.05571847507331378, 0.017653376507977415, 0.021361525635480466, 0.00020527521283798383, 0.0002622109452368169, 6.288034460609158e-05]
[1933, 0.9716666666666666]
[0, 0.0, 0.0013784943979825266, 0.0009899937741209865, 0.0, 1.5167962716643811e-05, 2.5629475961387892e-06]
[1934, 0.0]
[7.868832819804891, 0.2052401746724891, 0.06565381035738688, 0.07877786913725444, 0.0028240100943339543, 0.0013597739834241885, 0.0006248543761118012]
[1935, 1.0]
[0, 0.0, 0.01287542159652488, 0.005889291797890899, 0.0, 0.00477612124503194, 5.407633646737796e-05]
[1936, 0.05333333333333333]
[0.07383035914027619, 0.0004201680672268908, 0.002112136017771515, 0.0004713261318296198, 1.946131091390316e-06, 3.407923736864171e-05, 2.2952465915906765e-06]
[1493, 0.0]
[0.2783902056683989, 0.0039370078740157

[1950, 0.6583333333333334]
[2.5560981805379437, 0.013083048919226393, 0.03147478505160289, 0.005235355644267331, 4.362256993835941e-05, 0.00039784580154187706, 5.011957807725112e-06]
[1951, 0.24000000000000005]
[0.2035700216031806, 0.005934718100890208, 0.016394479936250733, 0.0015681073691250138, 0.00010735373054213633, 0.0006878064946193532, 6.413261585261684e-06]
[1952, 0.0]
[0, 0.0, 0.00014545724012144688, 0.00014697007575863684, 0.0, 3.872432568531227e-06, 6.940142015314606e-07]
[1953, 0.0]
Total time:1224.9714810848236
